In [146]:
import pandas as pd
import numpy as np
import pvlib
from pvlib.location import Location
import sympy as sm
from scipy import integrate

def inner(x1: sm.Matrix,x2: sm.Matrix):
    '''
    Computes the inner product of two vectors of same length.
    '''

    return x1.dot(x2, conjugate_convention = 'right')

sm.MutableDenseMatrix.inner = inner
sm.ImmutableDenseMatrix.inner = inner
L = 1.755
B = 1.038
maxFlux = 1000
maxEffect = 365/(L*B)
A = 0.5
S0 = 1100
S0A = S0 * A
effectPerFlux = maxEffect/maxFlux

# Metoder

In [27]:
def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = []
    for i in range(0,len(θ_s)):
        us = sm.Matrix([-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])])
        up = sm.Matrix([-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])])
        inprod = (inner(us,up))
        simpinprod = inprod.simplify()
        if(simpinprod < 0):
            results.append(0.0)
        else:
            results.append(simpinprod)
    return results

def fluxMinuteEntireDay(day, theta_p, phi_p):
    S0A=550
    L = 1.755
    B = 1.038

    zenithangs = np.deg2rad(np.array(solpos.loc[day].apparent_zenith))
    azimuthangs = np.deg2rad(np.array(solpos.loc[day].azimuth))
    invalidList = []
    for q in range(len(zenithangs)):
        if np.deg2rad(zenithangs[q])<0 or np.deg2rad(zenithangs[q])>(np.pi/2):
            invalidList.append(q)
    
    p_t = np.full(len(zenithangs),theta_p)
    p_p = np.full(len(zenithangs),phi_p)
 
    projList =  solar_panel_projection(zenithangs,azimuthangs,p_t,p_p)
    
    for index in invalidList:
        projList[index]=0
    for i in range(0,len(projList)):
        projList[i] = projList[i] * S0A

    
    return projList

def fluxHourEntireYear(theta_p, phi_p, zenithangs, azimuthangs):
    invalidList = []
    for q in range(len(zenithangs)):
        if (zenithangs[q])<0 or (zenithangs[q])>(np.pi/2):
            invalidList.append(q)
            print(np.rad2deg(zenithangs[q]))
    p_t = np.full(len(zenithangs),np.deg2rad(theta_p))
    p_p = np.full(len(zenithangs),np.deg2rad(phi_p))
 
    projList =  solar_panel_projection(zenithangs,azimuthangs,p_t,p_p)
    for index in invalidList:
        projList[index]=0

    for i in range(0,len(projList)):
        projList[i] = projList[i] * S0A

    
    return projList

def fluxToEffect(flux):
    result  = []
    for i in range (len(flux)): 
        result.append(effectPerFlux*flux[i] if flux[i]<=maxFlux else maxEffect)
       # result.append(flux[i]/maxFlux*maxEffect if flux[i]<=maxFlux else maxEffect)
    return result

def KwtPerDay(integral):
    L=1.755
    B=1.038
    J=integral*L*B
    return(J/3600000)

def NewFluxMethod(zenithangs,azimuthangs,i,j, i_list, j_list):
    p_t = np.full(len(zenithangs),np.deg2rad(theta_p))
    p_p = np.full(len(zenithangs),np.deg2rad(phi_p))
 
    projList =  solar_panel_projection(zenithangs,azimuthangs,p_t,p_p)
    for i in range(0,len(projList)):
        if(zenithangs[i] <= 0 or zenithangs[i] >= np.pi/2):
            projList[i] = 0
        else:
            projList[i] = effectPerFlux*(projList[i] * S0A) if (projList[i] * S0A) <= maxFlux else maxEffect
    
    return ((integrate.simpson(projList,dx = 3600))*L*B)/3600000


# Sol Værdier setup

In [28]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-04-01"
slut_dato = "2024-04-30"
delta_tid = "Min"  # "Min", "H",

valgt_dato = "2024-04-20"

# Definition of Location object. Coordinates and elevation of Amager, Copenhagen (Denmark)
site = Location(
    55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
)  # latitude, longitude, time_zone, altitude, name

# Definition of a time range of simulation
times = pd.date_range(
    start_dato + " 00:00:00", slut_dato + " 23:59:00", inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the 'Location' object
solpos = site.get_solarposition(times)

# Beregninger

## Flux værdier for theta_s = 45, phi_p = 180

In [29]:
fluxting =np.array((fluxMinuteEntireDay(valgt_dato, 45,180)))
for i in range(0,len(fluxting)):
    print(fluxting[i])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [30]:
integral = integrate.simpson(fluxToEffect(fluxting,1000,365),dx=60)
display(integral)

TypeError: fluxToEffect() takes 1 positional argument but 3 were given

## Fund af optimal vinkel d 20. april 2024, samt KwT og optimal vinkel

In [31]:
results = []
max = 0
angle = -1
for i in range(0,90):
        fluxting =np.array((fluxMinuteEntireDay(valgt_dato, i,180)))
        current = integrate.simpson(fluxToEffect(fluxting,1000,365),dx=60)
        current = KwtPerDay(current)
        results.append(current)
        if(current > max):
            max = current
            angle = i

print (max, "angle =",angle)

TypeError: fluxToEffect() takes 1 positional argument but 3 were given

## Fund af optimal vinkel over et år

In [153]:
def InInterval(f, start, end):
    for i in range(0,len(f)):
        if(f[i]>= end or f[i]<= start):
            f[i] = 0
    return f# Returnerer de to lister, en med funktionsværdierne og en med t værdierne 

def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = []
    for i in range(0,len(θ_s)):
        us = sm.Matrix([-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])])
        up = sm.Matrix([-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])])
        inprod = (inner(us,up))
        simpinprod = inprod.simplify()
        if(simpinprod < 0):
            results.append(0.0)
        else:
            results.append(simpinprod)
    return results


def NewFluxMethod(zenithangs,azimuthangs, i_list, j_list):
    projList =  solar_panel_projection(zenithangs,azimuthangs,i_list,j_list)
    for i in range(0,len(projList)):
        if(zenithangs[i] <= 0 or zenithangs[i] >= np.pi/2):
            projList[i] = 0
        else:
            projList[i] = effectPerFlux*(projList[i] * S0A) if (projList[i] * S0A) <= maxFlux else maxEffect
    
    return ((integrate.simpson(projList,dx = 3600))*L*B)/3600000

# Beregner zenith vinkel til elevationsvinkel
def solar_elevation_angledeg(theta):
    return 90 - theta

In [154]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-01-01"
slut_dato = "2024-12-31"
delta_tid = "H" # "Min", "H",
# Definition of Location object. Coordinates and elevation of DTU, Copenhagen (Denmark)
site = Location(
55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
) # latitude, longitude, time_zone, altitude, name
# Definition of a time range of simulation
times = pd.date_range(
start_dato + " 00:00:00", slut_dato + " 23:59:00", inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the ’Location’ object
solpos = site.get_solarposition(times)
results = []
#Lister af solens Zenith og Azimuth vinkler hver time over et  ̊ar
zenithangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].apparent_zenith))
azimuthangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].azimuth))
# Azimuth vinkel for panelet
phi_p = 180
p_p = np.full(len(zenithangs),np.deg2rad(phi_p))
#Der itereres over panelets zenith vinkler fra 0 til 90
for i in range(0,91):
    theta_p = i
    p_t = np.full(len(zenithangs),np.deg2rad(theta_p))
    # Effekten for et helt  ̊ar ved en given zenith vinkel for panelet beregnes
    effectPerYear = NewFluxMethod(zenithangs,azimuthangs, p_t, p_p)
    results.append(effectPerYear)
    print(effectPerYear)
    # Finder maks effekt og tilsvarende vinkel
max = 0
angle = -1
for i in range(0,len(results)):
    if(results[i] > max):
        max = results[i]
        angle = i
print("Energy Production is:",max,"Angle is:",solar_elevation_angledeg(angle),"Degrees from horizon",angle, "from zenith")

337.746819298796
343.982798487964
350.140093595000
356.219270962857
362.222248738471
368.160132146661
374.025372333645
379.795303931393
385.470762357039
391.053234566137
396.552343994288
401.960157531721
407.257386739656
412.443708950794
417.518417163369
422.481161381680
427.331752614201
432.070439427481
436.698103073751
441.216703450879
445.629732307310
449.951250357339
454.182376717503
458.283168715260
462.253288063318
466.092293901696
469.800440921769
473.377594751370
476.824988078101
480.143965722030
483.338848399032
486.425327915202
489.391137056506
492.208090065446
494.875306755847
497.392148027872
499.757829041139
501.971748052622
504.033457452069
505.942747661830
507.699826682028
509.305400920789
510.760411376991
512.065371123267
513.220426939766
514.225687853190
515.081217483094
515.787107753732
516.343620988278
516.751289910744
517.011297480092
517.126082173788
517.104305757781
516.931215776691
516.600663539651
516.112749736071
515.467622989228
514.665479810974
513.7065645418

In [13]:
print(results)

[752.653065932710 762.106939753692 771.346092533956 780.369332123676
 789.176454788873 797.768055901056 806.145762917356 814.314953560304
 822.291846630687 830.094208736556 837.645230594105 844.941577745482
 851.981356133421 858.762546146706 865.283228989925 871.541831179159
 877.537166720617 883.268436333367 888.735598992668 893.939151215116
 898.879361754878 903.555964720543 907.968731368585 912.116996947235
 916.000308368378 919.618257069321 922.971026729779 926.059637615101
 928.886866808066 931.465863653574 933.782207159975 935.814111560322
 937.560957916853 939.022214123244 940.197435066705 941.086262763544
 941.688426468241 942.003742755891 942.043412635167 941.834310506510
 941.360657317502 940.618010918137 939.604216323899 938.318303076779
 936.759793305281 934.928519551903 932.824365921542 930.447133801348
 927.796555803395 924.871697268384 921.670843988078 918.192709105858
 914.436913709981 910.403839585952 906.094302140898 901.509397564058
 896.650555591320 891.518940464467

In [14]:
max = 0
angle = -1
for i in range(0,len(results)):
    if(results[i] > max):
        max = results[i]
        angle = i
        
print(max,angle)

942.043412635167 38
